# Evaluate your machine-learning model

Authors: [Alexandre Gramfort](http://alexandre.gramfort.net), [Thomas Moreau](https://tommoral.github.io/about.html), and [Pedro L. C. Rodrigues](https://plcrodrigues.github.io/).

## Table of contents

* [1 The benefits of cross-calidation](#benefitscv)
    * [1.1 Load our dataset](#benefitscv_load)
    * [1.2 Empirical error vs generalization error](#benefitscv_empirical)
    * [1.3 A single error is not enough... what about the variance?](#benefitscv_single)
    * [1.4 Effect of the sample size on the variance analysis](#benefitscv_effect)
* [2 Comparing results versus baseline and chance level](#comparing)
* [3 Choice of cross-validation](#choicecv)

A core question in machine learning is how to evaluate the performance of a model once it's parameters are estimated (i.e. the model has been trained). In this notebook, we aim at presenting how you should answer this question in a statistically sound way. First, we will present the benefits of using cross-validation for this task and then have a quick look at different strategies and metrics that one should use in supervised learning.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 1 The benefits of cross-validation<a class="anchor" id="benefitscv"></a> [↑](#Table-of-contents)

## 1.1 Load our dataset<a class="anchor" id="benefitscv_load"></a> [↑](#Table-of-contents)

To illustrate our discussion, we will use the [California housing](https://inria.github.io/scikit-learn-mooc/python_scripts/datasets_california_housing.html) dataset. 

This dataset is a regression problem in which we want to estimate the median housing value given housing information.

In [ ]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
y *= 100

In [ ]:
X.head()

In [ ]:
y.head()

## 1.2 Empirical error vs. generalization error<a class="anchor" id="benefitscv_empirical"></a> [↑](#Table-of-contents)

Let's start by creating a regressor which will be a decision tree.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

regressor = DecisionTreeRegressor()

Let's first train our regressor on the full dataset.

In [ ]:
regressor.fit(X, y)

Now that our regressor is trained, we can check the regressor performance. For this purpose, we will use the mean absolute error which will give us an error in the native unit of the target, i.e. k$.

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred = regressor.predict(X)
score = mean_absolute_error(y_pred, y)
print(f"In average, our regressor make an error of {score:.2f} k$")

<div class="alert alert-success">
    <b>QUESTION</b>:
     <ul>
     <li>Are you surprised to get such performance with this regressor?</li>
     <li>Do you expect this regressor to perform this way in production?</li>
     </ul>
</div>

Surprisingly or not, our regressor is perfect. However, we are currently not able to confirm if our model would work on future unseen data. We can simulate this stage by splitting our dataset into two datasets and keep a dataset out of the learning process.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0
)

In [ ]:
?train_test_split

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
y_pred = regressor.predict(X_train)
score = mean_absolute_error(y_pred, y_train)
print(f"In average, our regressor make an error of {score:.2f} k$")

When evaluating our regressor on the training data, we still get a perfect model. This error is called the **empirical error**. Let see, if we get as lucky on the left-out dataset.

In [ ]:
y_pred = regressor.predict(X_test)
score = mean_absolute_error(y_pred, y_test)
print(f"In average, our regressor make an error of {score:.2f} k$")

... and not. By evaluating our model on a left-out set, we are checking that our model will be able to work on unseen data and thus if it is able to generalize. This type of error is called the **generalization error** and is the one that any data scientist hopes to decrease when creating a model.

<div class="alert alert-success">
    <b>QUESTION</b>:
     <ul>
     <li>Are we sure that our algorithm is robust?</li>
     </ul>
</div>

In [ ]:
for _ in range(5):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    regressor.fit(X_train, y_train)
    print(regressor.score(X_test, y_test))

<div class="alert alert-info">
    <h3>Generalization error:</h3>
     The aim of model training is to select the model $f$ out of a class of models $\mathcal F$ that minimizes a measure of the risk. The risk is measured with a loss $l$ between the true value $y$ associated to $x$ and the prediction $f(x)$ and thus we want to find:  
        $$
        f^\star = \arg\min_{f \in \mathcal F}\mathbb E_{(x, y) \sim \pi}[l(f(x), y]
        $$  
    The issue is that we cannot compute the expectation $\mathbb E_{(x, y) \sim \pi}$ because we don't know the input distribution $\pi$. Therefore, we approximate it with a set of examples $\{(x_1, y_1), \dots (x_N, y_N)\}$ drawn <i>i.i.d.</i> from $\pi$ and use the <b>Empirical Risk Minimization</b> (ERM):
        $$
        \widehat{f} = \arg\min_{f \in \mathcal F}\frac1N\sum_{i=1}^Nl(f(x_i), y_i]
        $$
    If the samples are drawn independently, we know that the error has a variance of $\mathcal O\left(\frac{1}{\sqrt{N}}\right)$. Thus there is a gap between the minimizer of the risk and the minimizer of the empirical risk. If we optimize too much for the ERM, the gap might be big and the selected model will have bad performance on unseen data. This is what is called <b>overfitting</b>. To control, this, one need to have a measure of the risk independent from the measure of the risk which is used to select the model: the Empirical Risk on the test set!
</div>


## 1.3 A single error is not enough... what about the variance?<a class="anchor" id="benefitscv_single"></a> [↑](#Table-of-contents)

While we were able to estimate the generalization error, we are indeed unable to know anything about the variance of our model and thus if it is robust or not. This is where the framework of cross-validation is used. Indeed, we can repeat our experiment and compute several time our generalization error and get intuition about the stability of our model.

The simplest way that we can think of is to shuffle our data and split it into two sets as we previously did and repeat several time our experiment. In scikit-learn, using the function `cross_validate` with the cross-validation `ShuffleSplit` allows us to make such evaluation.

In [ ]:
%%time
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ShuffleSplit

cv = ShuffleSplit(n_splits=30, test_size=0.2)

result_cv = cross_validate(
    regressor, X, y, cv=cv, scoring="neg_mean_absolute_error",
    n_jobs=6
)
result_cv

Once our cross-validation is done, we see that we got a Python dictionary with information regarding the cross-validation. Let's use a pandas dataframe to easily explore the content of the results.

In [ ]:
result_cv = pd.DataFrame(result_cv)
result_cv

We got information about the training and testing time which we can discard for the moment. However, we see that we got the `test_score` which is our generalization scores. We have 30 numbers because we repeated 30 times the same experiment by shuffling the data. Now, we can plot the distribution of these results.

In [ ]:
sns.displot(-result_cv["test_score"], kde=True)
_ = plt.xlabel("Mean absolute error (k$)")

We can observe that the generalization error is centered around 45.5 k\\$ and range from 44 k\\$ to 47 k\\$.

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
     <li>Can we conclude anything about the performance of our regressor?</li>
     <li>Can we conclude anything about our cross-validation analysis?</li>
     </ul>
    <b>Hint: </b>Plot the distribution of the target.
</div>

To be able to know if we can use our cross-validation results, we should put them into perspective with the problem that we try to solve. To know more about this, we can check the distribution of the original target (which we should have done beforehand).

In [ ]:
# %load solutions/solution_1.py

We see that the median value range from 50 k\\$ up to 500 k\\$. Thus an error range of 3 k\\$ means that our cross-validation results can be trusted and do not suffer from an execessive variance. Regarding the performance of our model itself, we can see that making an error of 45 k\\$ would be problematic even more if this happen for housing with low value. However, we also see some limitation regarding the metric that we are using. Making an error of 45 k\\$ for a target at 50 k\\$ and at 500 k\\$ should not have the same impact. We should instead use the mean absolute percentage error which will give a relative error.

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
     <li>Check the distribution of the training errors similarly to what we did for the generalization errors.</li>
     <li>What can you conclude?</li>
     </ul>
</div>

What about the empirical error?

In [ ]:
# %load solutions/solution_2.py

In [ ]:
# %load solutions/solution_3.py

## 1.4 Effect of the sample size on the variance analysis<a class="anchor" id="benefitscv_effect"></a> [↑](#Table-of-contents)

We are quite lucky. Our dataset count many samples.

In [ ]:
y.size

Let's make an experiment and reduce the number of samples and repeat the previous experiment.

In [ ]:
def make_cv_analysis(regressor, X, y):
    cv = ShuffleSplit(n_splits=10, test_size=0.2)
    result_cv = pd.DataFrame(
        cross_validate(
            regressor, X, y, cv=cv,
            scoring="neg_mean_absolute_error",
            n_jobs=-1
        )
    )
    return y.size, (result_cv["test_score"] * -1).values

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
     <li>Make an experiment by subsampling the dataset and plotting the distribution of the generalization errors.</li>
     <li>What can you conclude?</li>
     </ul>
</div>

In [ ]:
# %load solutions/solution_4.py

We see that with a low number of samples, the variance is much larger. Indeed, for low number of sample, we cannot even trust our cross-validation and therefore cannot conclude anything about our regressor. Therefore, it is really important to make experiment with a large enough sample size to be sure about the conclusions which would be drawn.

# 2 Comparing results versus baseline and chance level<a class="anchor" id="comparing"></a> [↑](#Table-of-contents)

Previously, we compare the generalization error by taking into account the target distribution. A good practice is to compare the generalization error with a dummy baseline and the chance level. In regression, we could use the `DummyRegressor` and predict the mean target without using the data. The chance level can be determined by permuting the labels and check the difference of result.

In [ ]:
from sklearn.dummy import DummyRegressor

dummy = DummyRegressor()
result_dummy = cross_validate(
    dummy, X, y, cv=cv, scoring="neg_mean_absolute_error",
    n_jobs=-1
)

In [ ]:
from sklearn.model_selection import permutation_test_score

score, permutation_score, pvalue = permutation_test_score(
    regressor, X, y, cv=cv, scoring="neg_mean_absolute_error",
    n_jobs=-1, n_permutations=10,
)

We plot the generalization errors for each of the experiment. We see that even our regressor does not perform well, it is far above chances our a regressor that would predict the mean target.

In [ ]:
final_result = pd.concat(
    [
        result_cv["test_score"] * -1,
        pd.Series(result_dummy["test_score"]) * -1,
        pd.Series(permutation_score) * -1,
    ], axis=1
).rename(columns={
    "test_score": "Cross-validation score",
    0: "Dummy score",
    1: "Permuted score",
})

In [ ]:
sns.displot(final_result, kind="kde")
_ = plt.xlabel("Mean absolute error (k$)")

# 3 Choice of cross-validation<a class="anchor" id="choicecv"></a> [↑](#Table-of-contents)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
     <li>Is `ShuffleSplit` cross-validation always the best cross-validation to use?</li>
     </ul>
</div>

Let's take an example of some financial quotes. These are the value of compagny  stocks with the time.

In [ ]:
import pandas as pd
import os
from urllib.request import urlretrieve

symbols = {'TOT': 'Total', 'XOM': 'Exxon', 'CVX': 'Chevron',
           'COP': 'ConocoPhillips', 'VLO': 'Valero Energy'}

quotes = pd.DataFrame()

for symbol, name in symbols.items():
    url = ('https://raw.githubusercontent.com/scikit-learn/examples-data/'
           'master/financial-data/{}.csv')
    filename = "data/{}.csv".format(symbol)
    if not os.path.exists(filename):
        urlretrieve(url.format(symbol), filename)
    this_quote = pd.read_csv(filename)
    quotes[name] = this_quote['open']
quotes.index = pd.to_datetime(this_quote['date'])

quotes.drop(["Total"], axis=1, inplace=True)

In [ ]:
_, ax = plt.subplots(figsize=(10, 6))
_ = quotes.plot(ax=ax)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

X, y = quotes.drop(columns=["Chevron"]), quotes["Chevron"]
regressor = GradientBoostingRegressor()

In [ ]:
cv = ShuffleSplit(n_splits=30)
result_cv = pd.DataFrame(
    cross_validate(
        regressor, X, y, cv=cv,
    )
)
print(f'Mean R2: {result_cv["test_score"].mean():.2f}')

<div class="alert alert-success">
    <b>QUESTION</b>:
     It seems that we have the perfect regressor. Does this seem normal to you?
</div>

In [ ]:
# %load solutions/solution_5.py

Let's check the different type of cross-validation that are available in scikit-learn:

https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html#sphx-glr-auto-examples-model-selection-plot-cv-indices-py